# Pokémon competition

In this notebook you have to provide the best pipeline that you have found to predict Pokémon battles.

At the end you will have to generate a set of predictions over the unlabeled data `data.hidden` and `data_inverse.hidden`. In these unlabeled dataset you will find all the Pokémon battles that we will be performing in some *fictional* Pokémon competition, so we do not know the outcome of these battles right now!

Remember to use all the tools that we have seen in class to evaluate and fine-tune your pipeline.

*Gotta Predict 'Em All!*

Paste here your pipeline:

In [ ]:
# Data: Replace it with the data from the provided datasets
X = [[1, 2]]
y = [1]

# Pipeline: Replace it with the pipeline created in the other notebook.
# You can remove the import after replacing the pipeline.
from sklearn.dummy import DummyClassifier
pipeline = DummyClassifier(strategy="uniform")

# Train the pipeline
pipeline.fit(X, y)

In [ ]:
#NOTE: WE HAD TO MODIFY THE DO NOT CHANGE THIS CODE PART BECAUSE
# IT WAS NOT IDENTICAL TO THE BASIC NOTEBOOK
# WE HAD TO ADD index_col=0 TO THE READ CSV METHOD

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!
# Do not change this code
# !!!!!!!!!!!!!!!!!!!!!!!!
import pandas
from pathlib import Path

__wd__ = Path("__file__").resolve().parent
datasets_path = __wd__ / "datasets"

tournament = pandas.read_csv(datasets_path / "data.hidden", index_col=0)
tournament_inverse = pandas.read_csv(datasets_path / "data_inverse.hidden", index_col=0)

y_predicted = pipeline.predict(tournament)
y_inverse_predicted = pipeline.predict(tournament_inverse)

y_predicted.tofile("predicted.csv", sep=",")
y_inverse_predicted.tofile("predicted_inverse.csv", sep=",")
# !!!!!!!!!!!!!!!!!!!!!!!!
# Do not change this code
# !!!!!!!!!!!!!!!!!!!!!!!!